Notebook which will:
- Obtain alert photometry for named SN and convert to AMPEL LightCurve object.
- Match with catalogs from the DESY match service.

In [ ]:
# Access to the AMPEL data archive assumes an individual *archive token* which can be obtained from 
# https://ampel.zeuthen.desy.de/live/dashboard/tokens
# This currently builds on GitHub authorization for AmpelProject members.
token = ""

In [ ]:
# Name of ZTF objects to lookup
snname = "ZTF22aabjpxh"

In [ ]:
import requests
from ampel.log.AmpelLogger import AmpelLogger
from ampel.ztf.util.ZTFIdMapper import ZTFIdMapper
from ampel.ztf.ingest.ZiDataPointShaper import ZiDataPointShaper
from ampel.content.T1Document import T1Document
from ampel.view.LightCurve import LightCurve
from ampel.ztf.t2.T2CatalogMatch import T2CatalogMatch

In [ ]:
def api_to_lightcurve(name, token, shaper=None):
    """
    Retrieve the alert history of a SN and convert to LightCurve
    
    """
    
    # Setup connection
    endpoint = "https://ampel.zeuthen.desy.de/api/ztf/archive/v3/object/{}/photopoints".format(name)
    header = {"Authorization": "bearer "+token}

    response = requests.get(endpoint, headers=header )
    
    if not response.ok:
        print('... failed to get alert')
        return None
    
    # Convert
    alert = response.json()
    if alert is None:
        print(' ... no alert content')
        return None
    
    pps = [alert['candidate']]
    pps.extend( [prv_cand for prv_cand in alert['prv_candidates'] ] )

    if shaper is None:
        tmplog = AmpelLogger.get_logger()
        shaper = ZiDataPointShaper(logger=tmplog)
    
    stockId = ZTFIdMapper.to_ampel_id(name)
    dps = shaper.process( pps, stockId)
    t1d = T1Document(stock=stockId, link=0)
    return LightCurve.build(t1d, dps)

In [ ]:
lc = api_to_lightcurve(snname, token)

In [ ]:
# The returned data has been shifted to a LightCurve view, the representation of a series of photometry
# available to e.g. T2 units in AMPEL. For more info:
# https://github.com/AmpelProject/Ampel-photometry/blob/master/ampel/view/LightCurve.py
# For example, to return the date, magnitue and magnitude error in the ZTF r filter (fid=2):
lc.get_ntuples(["jd", "magpsf", "sigmapsf"], {'attribute': 'fid', 'operator': '==', 'value': 2})

We will additionally use the online catalog matching service to check for information in NED.

In [ ]:
# Match to the DESY ned copy
catconfig = {
        'NEDz' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
        }, }

In [ ]:
# Initalize
logger = AmpelLogger.get_logger()
t2cat = T2CatalogMatch( catalogs = catconfig,
    resource = {"ampel-ztf/catalogmatch":'https://ampel.zeuthen.desy.de/api/catalogmatch/'},
    logger=logger)

In [ ]:
# Get a photopoint from which to get coordinates 
t2cat_out = t2cat.process(lc.photopoints[0])

In [ ]:
t2cat_out